<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

COMMON_PATH = '/workspace/common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "30g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [4]:
df = df.sample(False, 0.5)

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [7]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model'))

In [8]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

1832013

In [9]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [10]:
!pip3.5 install hyperopt

     |████████████████████████████████| 1.9 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 48.6 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 7.6 MB/s  eta 0:00:01
     |████████████████████████████████| 829 kB 47.5 MB/s eta 0:00:01
    Running setup.py install for networkx ... done
    Running setup.py install for future ... done


## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [11]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [12]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 1
}

Fix baseline parameters and train baseline model

In [13]:
CONTROL_NAME = 'xgb baseline'

baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [14]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.724982372665222

In [15]:
all_metrics = {}

In [16]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [17]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.15, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 50.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 20, 'max_depth': 6, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5106030428887525, ROC-AUC: 0.7249823726652225
SPACE:                                                                          
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.1, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 50.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 6, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5045517286307888, ROC-AUC: 0.7328327427168098                       
SPACE:                                                                          
{'eval_metric': 'logloss',

LOG-LOSS: 0.5046078213299982, ROC-AUC: 0.7324786516543903                        
SPACE:                                                                           
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.5, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 50.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 20, 'max_depth': 6, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5055290484861853, ROC-AUC: 0.7308328106158284                        
SPACE:                                                                           
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.1, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 50.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 10, 'max_depth': 6, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5441330185926748, ROC-AUC: 0.718502029832048

In [18]:
best

{'eta': 4, 'num_round': 3}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [19]:
eta = eta_choice[best['eta']]
num_round = num_round_choice[best['num_round']]

In [22]:
print(eta, num_round)

0.3 100


### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

#### Tune max_depth, min_child_weight

In [23]:
%%time

max_depth_choice = [4, 5, 6, 8]
min_child_weight_choice = [30.0, 50.0, 70.0, 90.0]

space = {
    # Optimize
    'max_depth': hp.choice('max_depth', max_depth_choice),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),

    # Fixed
    'eta': eta,
    'num_round': num_round,
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 70.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 6, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5013970013993823, ROC-AUC: 0.7369639635122962
SPACE:                                                                           
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 30.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 5, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5024510772492848, ROC-AUC: 0.7355346200939645                        
SPACE:                                                                           
{'eval_metric': 'loglos

LOG-LOSS: 0.5026277872796509, ROC-AUC: 0.7352302326283091                         
SPACE:                                                                            
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 90.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 8, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5004902555711931, ROC-AUC: 0.7380858564448036                         
SPACE:                                                                            
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 70.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 5, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5025534034699499, ROC-AUC: 0.735281839

In [24]:
max_depth = max_depth_choice[best['max_depth']]
min_child_weight = min_child_weight_choice[best['min_child_weight']]

In [25]:
print(max_depth, min_child_weight)

8 90.0


#### Tune gamma

In [26]:
%%time

gamma_choice = [0.3, 0.5, 0.7, 0.9, 1.0]


space = {
    # Optimize
    'gamma': hp.choice('gamma', gamma_choice),

    # Fixed
    'eta': eta,
    'num_round': num_round,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': baseline_params['subsample'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.7, 'min_child_weight': 90.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 8, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.500480963462351, ROC-AUC: 0.7380581654050171
SPACE:                                                                          
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.3, 'min_child_weight': 90.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 8, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5003227847414673, ROC-AUC: 0.7383871795775682                       
SPACE:                                                                           
{'eval_metric': 'logloss',

LOG-LOSS: 0.5004902555711931, ROC-AUC: 0.7380858564448005                         
SPACE:                                                                            
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.9, 'min_child_weight': 90.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 8, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5004902555711931, ROC-AUC: 0.738085856444809                          
SPACE:                                                                            
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 1.0, 'min_child_weight': 90.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 8, 'silent': 0, 'subsample': 0.9, 'nworkers': 1}
LOG-LOSS: 0.5002968977085706, ROC-AUC: 0.738420262

In [27]:
gamma = gamma_choice[best['gamma']]  # change me!

In [28]:
print(eta, num_round, max_depth, min_child_weight, gamma)

0.3 100 8 90.0 0.5


#### Tune subsample, colsample_bytree

In [ ]:
%%time

subsample_choice = [0.5, 0.7, 0.9, 1.0]
colsample_bytree_choise = [0.5, 0.7, 0.9, 1.0]


space = {
    # Optimize
    'subsample': hp.choice('subsample', subsample_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choise),

    # Fixed
    'gamma': gamma,
    'eta': eta,
    'num_round': num_round,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.5, 'min_child_weight': 90.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 0.9, 'num_round': 100, 'max_depth': 8, 'silent': 0, 'subsample': 0.7, 'nworkers': 1}
LOG-LOSS: 0.5004370006321788, ROC-AUC: 0.7381655528058519
SPACE:                                                                           
{'eval_metric': 'logloss', 'predictionCol': 'prediction', 'eta': 0.3, 'nthread': 1, 'gamma': 0.5, 'min_child_weight': 90.0, 'objective': 'binary:logistic', 'featuresCol': 'features', 'labelCol': 'label', 'colsample_bytree': 1.0, 'num_round': 100, 'max_depth': 8, 'silent': 0, 'subsample': 0.5, 'nworkers': 1}
LOG-LOSS: 0.5005157048468959, ROC-AUC: 0.737957299556895                         
SPACE:                                                                           
{'eval_metric': 'loglos

In [ ]:
subsample = subsample_choice[best['subsample']]
colsample_bytree = colsample_bytree_choise[best['colsample_bytree']]

print(eta, num_round, max_depth, min_child_weight, gamma, subsample, colsample_bytree)

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [ ]:
%%time

reg_lambda_choice = [0.5, 0.7, 0.9, 1.0]
alpha_choise = [0.5, 0.7, 0.9, 1.0]


space = {
    # Optimize
    'reg_lambda': hp.choice('reg_lambda', reg_lambda_choice),
    'alpha': hp.choice('alpha', alpha_choise),

    # Fixed
    'colsample_bytree': colsample_bytree,
    'subsample': subsample,
    'gamma': gamma,
    'eta': eta,
    'num_round': num_round,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

In [ ]:
reg_lambda = lambda_choice[best['lambda']]
alpha = colsample_bytree_choise[best['alpha']]

print(eta, num_round, max_depth, min_child_weight, gamma, subsample, colsample_bytree, reg_lambda, alpha)

### Lower the learning rate and decide the optimal parameters

In [ ]:
optimal_params = {
    'reg_lambda': reg_lambda,
    'alpha': alpha,
    'colsample_bytree': colsample_bytree,
    'subsample': subsample,
    'gamma': gamma,
    'eta': eta,
    'num_round': num_round,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
}

optimal_params

---
## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте справнение на отложенной выборке

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

Итоговая таблица

In [23]:
get_ate(all_metrics, CONTROL_NAME)

,metric,xgb opt ate %
0,logloss,0.000000e+00
1,rocauc,-1.554312e-13
